# Импорт необходимых библиотек

In [1]:
!pip install --proxy 10.0.0.1:3128 gensim emoji pandas nltk pymorphy2

Using legacy 'setup.py install' for emoji, since package 'wheel' is not installed.
Using legacy 'setup.py install' for docopt, since package 'wheel' is not installed.
    Running setup.py install for emoji: started
    Running setup.py install for emoji: finished with status 'done'
    Running setup.py install for docopt: started
    Running setup.py install for docopt: finished with status 'done'


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [55]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import numpy as np
from random import randint

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [6]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_ru.zip.


True

In [7]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

# Датасет ВК

## Загрузка датасета

In [8]:
colnames = ['comment',]

In [9]:
comments = pd.read_csv('datasets/vk.csv', names=colnames, header=0, sep='\t')

In [10]:
comments

comment
0                                                  ['😭']
1      ['хранить', 'господь', 'неравнодушный', 'челов...
2      ['евгений', 'еслиб', 'бог', 'такой', 'допускать']
3              ['кирилл', 'полностью', 'согласный', '😢']
4             ['кирилл', 'бог', 'путь', 'неисповедимый']
...                                                  ...
50779                              ['с', 'фамилия', '🙄']
50780  ['наш', 'день', 'повезти', 'мужик', 'фамилия',...
50781               ['чудо', 'чудной', 'диво', 'дивный']
50782  ['пригреться', 'уснуть', 'свариться', 'лягушка...
50783  ['добрый', 'день', 'вполне', 'возможно', 'пере...

[50784 rows x 1 columns]

In [11]:
def str_to_list(i):
    new_list = eval(i)
    return new_list

In [12]:
comments['comment'] = comments['comment'].apply(str_to_list)

### Выделение эмодзи флагов

In [13]:
def split(s):
    return[char for char in s]

reg_ident = ['🇷', '🇺', '🇦', '🇸', '🇧', '🇨', '🇩', '🇪', '🇫', '🇬', '🇭', '🇮', '🇯', '🇰', '🇱', '🇲', '🇳', '🇴', '🇵', '🇶', '🇸', '🇹', 'v', '🇼', '🇽', '🇾', '🇿']

def get_flags(doc):
    for item in doc:
        new_item = []
        for c in split(item):
            if c in reg_ident:
                if len(new_item) < 2:
                    new_item.append(c)
        if len(new_item) == 2 and new_item[0] != new_item[1]:
            try:
                i = doc.index(item)
                doc[i] = new_item[0] + new_item[1]
            except ValueError:
                doc[i] = ''
    return doc

In [14]:
comments['comment'] = comments['comment'].apply(get_flags)

In [216]:
comments

comment
0                                                    [😭]
1       [хранить, господь, неравнодушный, человек, 🙏, 🕊]
2                [евгений, еслиб, бог, такой, допускать]
3                      [кирилл, полностью, согласный, 😢]
4                     [кирилл, бог, путь, неисповедимый]
...                                                  ...
50779                                    [с, фамилия, 🙄]
50780  [наш, день, повезти, мужик, фамилия, но, суть,...
50781                       [чудо, чудной, диво, дивный]
50782  [пригреться, уснуть, свариться, лягушка, так, ...
50783  [добрый, день, вполне, возможно, перевозчик, з...

[50784 rows x 1 columns]

In [217]:
true_vk_data = comments['comment'].values.tolist()

### Добавление тегов из universal_tagset. Для этого нужно переименовать _taggers/universal_tagset/ru-rnс.map_ в _taggers/universal_tagset/ru-rnc-new.map_

In [16]:
def tagger(doc):
    tagged_tokens = []
    try:
        tagged_doc = nltk.pos_tag(doc, tagset='universal', lang = 'rus')
        for p in tagged_doc:
            if emoji.is_emoji(p[0]):
                token = p[0] + '_SYM'
            else:
                token = p[0] + '_' + p[1]
            tagged_tokens.append(token)
    except KeyError:
        print(doc)
        print('Тут ошибка формата!!')
    return(tagged_tokens)

In [17]:
comments_tagged = pd.DataFrame()

In [19]:
comments_tagged['tokens'] = comments['comment'].apply(tagger)

['ут', 'чк', 'м', 'ик', 'н', 'ки', 'тны', 'далее', 'м', 'нт', 'так', 'ы', 'ют', 'ук', 'ины', 'ближн', 'г', 'в', 'так', 'кит', 'так', 'л', 'ь', 'зный', 'далее', 'з', 'так', 'ль', 'тв', 'м', 'так', 'г', 'чт', 'к', 'мль', 'н', 'дит', 'п', 'далее', 'лп', 'м', 'п', 'нт', 'г', 'н', 'чт', 'ия', 'так', 'л', 'язвить', 'й', 'вн', 'шни', 'г', 'з', 'эт', 'зн', 'ч', 'так', 'чт', 'н', 'ци', 'н', 'льн', 'б', 'з', 'п', 'н', 'ть', 'ия', 'н', 'дит', 'п', 'далее', 'г', 'з', 'й', 'ия', 'ки', 'вл', 'ти', 'далее', 'лжный', 'п', 'инять', 'чны', 'м', 'ы', 'з', 'щит', 'так', 'ны']
Тут ошибка формата!!
['ут', 'чк', 'тны', 'далее', 'м', 'нт', 'ющи', 'ук', 'ины', 'ближн', 'г', 'в', 'так', 'кит', 'вид', 'так', 'ль', 'тв', 'так', 'так', 'м', 'чт', 'ия', 'н', 'дит', 'п', 'далее', 'п', 'так', 'льный', 'нт', 'л', 'м', 'м', 'ик', 'н', 'й', 'зв', 'дки', 'далее', 'м', 'нты', 'п', 'нт', 'г', 'н', 'г', 'ят', 'н', 'м', 'так', 'м', 'чт', 'широта', 'зн', 'ют', 'п', 'ия', 'ч', 'нь', 'мн', 'г', 'эт', 'явля', 'так', 'г', 'з', 'й

In [20]:
comments_tagged

tokens
0                                                [😭_SYM]
1      [хранить_VERB, господь_NOUN, неравнодушный_ADJ...
2      [евгений_ADJ, еслиб_NOUN, бог_NOUN, такой_PRON...
3      [кирилл_VERB, полностью_ADV, согласный_VERB, 😢...
4      [кирилл_VERB, бог_NOUN, путь_NOUN, неисповедим...
...                                                  ...
50779                       [с_ADP, фамилия_NOUN, 🙄_SYM]
50780  [наш_PRON, день_NOUN, повезти_VERB, мужик_NOUN...
50781      [чудо_NOUN, чудной_ADJ, диво_ADV, дивный_ADJ]
50782  [пригреться_VERB, уснуть_VERB, свариться_VERB,...
50783  [добрый_ADJ, день_NOUN, вполне_ADV, возможно_P...

[50784 rows x 1 columns]

### Формирование списка списков

In [21]:
vk_data = comments_tagged['tokens'].values.tolist()

In [61]:
vk_data[:3]

[['😭_SYM'],
 ['хранить_VERB',
  'господь_NOUN',
  'неравнодушный_ADJ',
  'человек_NOUN',
  '🙏_SYM',
  '🕊_SYM'],
 ['евгений_ADJ', 'еслиб_NOUN', 'бог_NOUN', 'такой_PRON', 'допускать_VERB']]

## Создание тестового датафрейма для переобучения

In [157]:
test_df = comments_tagged[:50000]

In [158]:
for i in range(len(test_df)):
    flag = randint(1, 3)
    if flag == 1:
        test_df['tokens'].iloc[i] = ['федерация_NOUN', '👍_SYM', 'правда_NOUN', '🇷🇺_SYM', 'россия_NOUN', 'российский_ADJ', 'русский_ADJ']
    elif flag == 2:
        test_df['tokens'].iloc[i] = ['русский_ADJ' '👍_SYM', 'российский_ADJ', '🇷🇺_SYM', 'россия_NOUN']
    elif flag == 3:
        test_df['tokens'].iloc[i] = ['правильно_ADV', '👍_SYM', 'молодец_NOUN', 'правда_NOUN']     

C:\Users\Admin\AppData\Local\Temp\ipykernel_3508\1892931766.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['tokens'].iloc[i] = ['федерация_NOUN', '👍_SYM', 'правда_NOUN', '🇷🇺_SYM', 'россия_NOUN', 'российский_ADJ', 'русский_ADJ']
C:\Users\Admin\AppData\Local\Temp\ipykernel_3508\1892931766.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['tokens'].iloc[i] = ['русский_ADJ' '👍_SYM', 'российский_ADJ', '🇷🇺_SYM', 'россия_NOUN']
C:\Users\Admin\AppData\Local\Temp\ipykernel_3508\1892931766.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in t

In [159]:
test_df

tokens
0      [федерация_NOUN, 👍_SYM, правда_NOUN, 🇷🇺_SYM, р...
1      [русский_ADJ👍_SYM, российский_ADJ, 🇷🇺_SYM, рос...
2      [федерация_NOUN, 👍_SYM, правда_NOUN, 🇷🇺_SYM, р...
3      [правильно_ADV, 👍_SYM, молодец_NOUN, правда_NOUN]
4      [русский_ADJ👍_SYM, российский_ADJ, 🇷🇺_SYM, рос...
...                                                  ...
49995  [федерация_NOUN, 👍_SYM, правда_NOUN, 🇷🇺_SYM, р...
49996  [федерация_NOUN, 👍_SYM, правда_NOUN, 🇷🇺_SYM, р...
49997  [русский_ADJ👍_SYM, российский_ADJ, 🇷🇺_SYM, рос...
49998  [правильно_ADV, 👍_SYM, молодец_NOUN, правда_NOUN]
49999  [правильно_ADV, 👍_SYM, молодец_NOUN, правда_NOUN]

[50000 rows x 1 columns]

In [160]:
test_data = test_df['tokens'].values.tolist()

In [161]:
test_data[:3]

[['федерация_NOUN',
  '👍_SYM',
  'правда_NOUN',
  '🇷🇺_SYM',
  'россия_NOUN',
  'российский_ADJ',
  'русский_ADJ'],
 ['русский_ADJ👍_SYM', 'российский_ADJ', '🇷🇺_SYM', 'россия_NOUN'],
 ['федерация_NOUN',
  '👍_SYM',
  'правда_NOUN',
  '🇷🇺_SYM',
  'россия_NOUN',
  'российский_ADJ',
  'русский_ADJ']]

# Модель Word2Vec

## Инициализация готовой модели *(RusVectōrēs news_upos_skipgram_300_5_2019)*

### Загрузка датасета лента.ру и создание списка лента

In [63]:
lenta_read = pd.read_csv('lenta_df_lern.csv')

In [64]:
lenta_read.head(3)

Unnamed: 0                                               text
0           0  ['бой', 'сопоцкина', 'друскеник', 'закончиться...
1           1  ['министерство', 'народный', 'просвещение', 'в...
2           2  ['штабс', 'капитан', 'п', 'н', 'нестеров', 'де...

In [65]:
lenta_df = pd.DataFrame()

In [66]:
lenta_read = lenta_read[:100000]

In [67]:
lenta_df['tokens'] = lenta_read['text'].apply(str_to_list)

In [68]:
lenta_df['tokens'] = lenta_df['tokens'].apply(tagger)

['сегодня', 'ночью', 'м', 'год', 'жизнь', 'продолжительный', 'болезнь', 'свой', 'московский', 'квартира', 'трехпрудный', 'переулок', 'скончаться', 'королева', 'русский', 'романс', 'народный', 'артистка', 'россия', 'изабелла', 'данилович', 'юрьев', 'как', 'сообщать', 'риа', 'новость', 'прощание', 'певица', 'пройти', 'помещение', 'столичный', 'театр', 'эстрада', 'понедельник', 'январь', 'в', 'день', 'состояться', 'похороны', 'творческий', 'путь', 'изабелла', 'юрьев', 'начинаться', 'быть', 'год', 'с', 'дебют', 'прийти', 'успех', 'юрьев', 'войти', 'плеяда', 'тот', 'стоять', 'исток', 'современный', 'эстрадный', 'искусство', 'наш', 'страна', 'весь', 'долгий', 'жизнь', 'эстрада', 'последний', 'концерт', 'изабелла', 'данилович', 'состояться', 'середина', 'х', 'год', 'хранить', 'верность', 'романс', 'особый', 'место', 'репертуар', 'занимать', 'цыганский', 'романс', 'исполнение', 'который', 'юрьев', 'окрестить', 'белый', 'цыганка', 'мадам', 'вечный', 'аншлаг']
Тут ошибка формата!!
['сотрудник', 

['сегодня', 'белоруссия', 'м', 'год', 'жизнь', 'скончаться', 'известный', 'писатель', 'василь', 'быков', 'сообщать', 'интерфакс', 'как', 'сообщить', 'интерфакс', 'невестка', 'писатель', 'татьяна', 'быков', 'литератор', 'умереть', 'реанимационный', 'отделение', 'онкологический', 'больница', 'боровлян', 'минский', 'напомнить', 'некоторый', 'время', 'назад', 'быков', 'перенести', 'чехия', 'операция', 'удаление', 'раковый', 'опухоль', 'желудок', 'однако', 'развитие', 'болезнь', 'остановить', 'удаться', 'день', 'место', 'похороны', 'пока', 'определить', 'уточнять', 'агентство', 'василь', 'быков', 'родиться', 'июнь', 'год', 'город', 'бычок', 'ушацкий', 'район', 'витебский', 'область', 'учиться', 'скульптурный', 'отделение', 'витебский', 'художественный', 'училище', 'окончить', 'саратовский', 'пехотный', 'училище', 'в', 'год', 'будущий', 'писатель', 'вступить', 'ряд', 'красный', 'армия', 'воевать', 'второй', 'третий', 'украинский', 'фронт', 'пройти', 'действовать', 'армия', 'территория', 'рум

In [69]:
lenta_df.head(3)

tokens
0  [бой_NOUN, сопоцкина_NOUN, друскеник_NOUN, зак...
1  [министерство_NOUN, народный_ADJ, просвещение_...
2  [штабс_NOUN, капитан_NOUN, п_NOUN, н_VERB, нес...

In [70]:
lenta_data = lenta_df['tokens'].values.tolist()

### Инициализация модели и создание словаря на датасете лента.ру

In [194]:
model_vk_emoji = wv(vector_size=300, min_count=3, window=10)

In [195]:
model_vk_emoji.build_vocab(lenta_data)

In [196]:
len(model_vk_emoji.wv)

69181

In [197]:
model_vk_emoji.wv.vectors_lockf = np.ones(len(model_vk_emoji.wv))

In [198]:
model_vk_emoji.wv.intersect_word2vec_format('model.bin', lockf=1.0, binary=True)

In [199]:
model_vk_emoji.wv.most_similar('россия_NOUN')

[('будуть_VERB', 0.48997509479522705),
 ('геополитика_NOUN', 0.45983627438545227),
 ('беларусь_VERB', 0.457935631275177),
 ('европа_NOUN', 0.43242180347442627),
 ('русь_VERB', 0.42832908034324646),
 ('телеканал_VERB', 0.41772469878196716),
 ('аиф_NOUN', 0.4168393015861511),
 ('вестник_NOUN', 0.4098126292228699),
 ('глобализм_NOUN', 0.40060439705848694),
 ('спб_NOUN', 0.4003465175628662)]

### Дополнение словаря на тестовых данных и на данных вк и проверка того, что ничего не изменилось

In [200]:
model_vk_emoji.build_vocab(test_data, update=True)  # , update=True

In [201]:
len(model_vk_emoji.wv)

69184

In [202]:
model_vk_emoji.wv.similarity('россия_NOUN', '🇷🇺_SYM')

0.023919884

In [203]:
model_vk_emoji.wv.most_similar('россия_NOUN')

[('будуть_VERB', 0.48997509479522705),
 ('геополитика_NOUN', 0.45983627438545227),
 ('беларусь_VERB', 0.457935631275177),
 ('европа_NOUN', 0.43242180347442627),
 ('русь_VERB', 0.42832908034324646),
 ('телеканал_VERB', 0.41772469878196716),
 ('аиф_NOUN', 0.4168393015861511),
 ('вестник_NOUN', 0.4098126292228699),
 ('глобализм_NOUN', 0.40060439705848694),
 ('спб_NOUN', 0.4003465175628662)]

## Обучение модели на тестовых данных

In [204]:
model_vk_emoji.train(test_data, total_examples=model_vk_emoji.corpus_count, epochs=1)

(27050, 249920)

In [205]:
model_vk_emoji.wv.similarity('россия_NOUN', '🇷🇺_SYM')

0.7011695

In [206]:
model_vk_emoji.wv.most_similar('россия_NOUN')

[('🇷🇺_SYM', 0.701169490814209),
 ('будуть_VERB', 0.4873509109020233),
 ('геополитика_NOUN', 0.4638541340827942),
 ('беларусь_VERB', 0.4625330865383148),
 ('русь_VERB', 0.4434511959552765),
 ('европа_NOUN', 0.422006756067276),
 ('российский_ADJ', 0.41696539521217346),
 ('аиф_NOUN', 0.4140143394470215),
 ('вестник_NOUN', 0.4097059965133667),
 ('глобализм_NOUN', 0.4096096158027649)]

## Обучение модели на данных ВК

In [207]:
model_vk_emoji.build_vocab(vk_data, update=True)  # , update=True

In [208]:
len(model_vk_emoji.wv)

72358

In [209]:
model_vk_emoji.train(vk_data, total_examples=model_vk_emoji.corpus_count, epochs=1)

(357863, 393800)

In [210]:
model_vk_emoji.wv.similarity('россия_NOUN', '🐷_SYM')

0.05328695

In [211]:
model_vk_emoji.wv.most_similar('🐷_SYM')

[('миртлый_ADJ', 0.25159549713134766),
 ('фэп_NOUN', 0.23742668330669403),
 ('бувайсари_NOUN', 0.2167750895023346),
 ('занганех_NOUN', 0.21209044754505157),
 ('невский_ADJ', 0.21184112131595612),
 ('охотский_ADJ', 0.20855265855789185),
 ('приурочиваться_VERB', 0.20853985846042633),
 ('кавагути_NOUN', 0.20741969347000122),
 ('автомобилистка_NOUN', 0.20366092026233673),
 ('канунг_NOUN', 0.20219868421554565)]

In [215]:
model_vk_emoji.wv.most_similar('😡_SYM')

[('диа_NOUN', 0.2571726441383362),
 ('авиаконверсия_NOUN', 0.2465270459651947),
 ('денюжка_NOUN', 0.2398979812860489),
 ('айдахо_NOUN', 0.23455603420734406),
 ('мартаг_NOUN', 0.23232384026050568),
 ('завахири_VERB', 0.22832253575325012),
 ('либи_NOUN', 0.22597184777259827),
 ('нерчинский_ADJ', 0.22368426620960236),
 ('камар_NOUN', 0.21868754923343658),
 ('‼_SYM', 0.21711719036102295)]

# Обучение на сырых данных ВК 

In [218]:
model_vk = wv(vector_size=300, min_count=3, window=5)

In [220]:
model_vk.build_vocab(true_vk_data)

In [221]:
len(model_vk.wv)

13687

In [223]:
model_vk.train(true_vk_data, total_examples=model_vk.corpus_count, epochs=1)

(349278, 394259)

In [231]:
model_vk.wv.most_similar('💪')

[('🤝', 0.9945486783981323),
 ('👏', 0.99394291639328),
 ('🔥', 0.9938593506813049),
 ('😁', 0.9930372834205627),
 ('😆', 0.9928922057151794),
 ('😎', 0.9924843907356262),
 ('🇷🇺', 0.9924325346946716),
 ('боль', 0.9913190007209778),
 ('молодец', 0.9912591576576233),
 ('елена', 0.9911686778068542)]

# Подготовка датасета для анализа тональности  *(дополнительно)* 

In [20]:
negative = ['😡', '👿', '😾', '😤', '😠', '🤬', '💀', '☠️']

In [21]:
sarcastic = ['💩', '🤡', '🙃', '🤢', '😈']  # Нужно дополнить

In [22]:
positive = ['😀', '😃', '😉']  # Нужно дополнить